In [ ]:
%reload_ext autoreload
%autoreload
import sys
sys.path.append("../src")

In [ ]:
import pandas as pd
import json
import csv
from tqdm import tqdm

from sapsam import parser, constants
from sapsam.SignavioConventionsChecker import bp_conventions_checker, syntax_checker

# Read data

Let's first read in the meta data of all models in the dataset.

In [ ]:
df_meta = parser.parse_model()
df_meta.head()

# Checking conventions

In [ ]:
df_meta = df_meta[df_meta['namespace'] == 'http://b3mn.org/stencilset/bpmn2.0#']

In [ ]:
guideline_id = '4551c2229baa4c79a151b5a0cc1010d2'

In [ ]:
with open(constants.DATA_RAW / 'conventions.csv', mode='a', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['Model ID', 'Name', 'Syntax Errors', 'BP Violations Count'])
    if file.tell() == 0:
        writer.writeheader()
    print('Running conventions...')
                            
    for model_id in tqdm(df_meta.index.unique()):
        model_data = df_meta.loc[model_id]
        name = model_data['name']
        model_json = model_data['model_json']
        syntax_errors = syntax_checker(model_json)
        violations_count = bp_conventions_checker(name, model_id, guideline_id, model_json)
        writer.writerow({'Model ID': model_id, 'Name': name, 'Syntax Errors': syntax_errors, 'BP Violations Count': violations_count})

# Data analysis